In [1]:
import pandas as pd

In [26]:
fips_state = pd.read_csv("data/fips-by-state.csv", encoding='unicode_escape')
fips_county = pd.read_csv("data/us-state-ansi-fips.csv")
fips_county.columns = fips_county.columns.str.replace(' ', '')
fips_county['stusps'] = fips_county['stusps'].str.strip()
fips_county['stname'] = fips_county['stname'].str.strip()
pm = pd.read_csv("data/pm_conc_data.csv")

In [30]:
merged_index = pd.merge(fips_state, fips_county, left_on="state", right_on="stusps").drop("stusps", axis=1)
merged_index

,fips,name,state,stname,st
0,1001,Autauga County,AL,Alabama,1
1,1003,Baldwin County,AL,Alabama,1
2,1005,Barbour County,AL,Alabama,1
3,1007,Bibb County,AL,Alabama,1
4,1009,Blount County,AL,Alabama,1
...,...,...,...,...,...
3138,56037,Sweetwater County,WY,Wyoming,56
3139,56039,Teton County,WY,Wyoming,56
3140,56041,Uinta County,WY,Wyoming,56
3141,56043,Washakie County,WY,Wyoming,56


In [33]:
ca_counties = merged_index[merged_index['state'] == 'CA']
ca_counties

,fips,name,state,stname,st
186,6001,Alameda County,CA,California,6
187,6003,Alpine County,CA,California,6
188,6005,Amador County,CA,California,6
189,6007,Butte County,CA,California,6
190,6009,Calaveras County,CA,California,6
191,6011,Colusa County,CA,California,6
192,6013,Contra Costa County,CA,California,6
193,6015,Del Norte County,CA,California,6
194,6017,El Dorado County,CA,California,6
195,6019,Fresno County,CA,California,6


In [45]:
ca_pm_by_county = pm[pm['countyfips'].isin(ca_counties.fips)]
ca_pm_by_county = pd.merge(left=ca_pm_by_county, right=ca_counties[["fips", "name"]], left_on="countyfips", right_on="fips")
ca_pm_by_county

,Unnamed: 0,year,date,statefips,countyfips,ctfips,latitude,longitude,ds_pm_pred,ds_pm_stdd,fips,name
0,25,2014,12SEP2014,6,6033,6033000900,38.96380,-122.74283,12.9049,7.2206,6033,Lake County
1,100,2014,12SEP2014,6,6033,6033000702,38.97411,-122.50930,12.6224,6.9395,6033,Lake County
2,156,2014,12SEP2014,6,6033,6033001000,38.98325,-122.88611,12.8296,7.2218,6033,Lake County
3,182,2014,12SEP2014,6,6033,6033000701,38.98590,-122.64803,12.7661,7.7694,6033,Lake County
4,534,2014,12SEP2014,6,6033,6033000300,39.04385,-122.96437,12.0626,6.6618,6033,Lake County
...,...,...,...,...,...,...,...,...,...,...,...,...
1072,49939,2014,13SEP2014,6,6065,6065043242,33.59038,-117.10507,14.0963,7.7131,6065,Riverside County
1073,49949,2014,13SEP2014,6,6065,6065043227,33.59124,-117.22480,16.4971,8.9183,6065,Riverside County
1074,49985,2014,13SEP2014,6,6065,6065046200,33.59949,-114.59670,2.0796,1.1368,6065,Riverside County
1075,49990,2014,13SEP2014,6,6065,6065043271,33.60066,-117.27565,16.9781,9.3345,6065,Riverside County


In [51]:
ca_pm_by_county.shape[0]

1077

In [57]:
ca_pm_by_county.countyfips.value_counts().index.__len__()

26